In [ ]:
# Importar las bibliotecas necesarias
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,count,desc,sum,to_date,date_format,concat_ws,format_string,to_timestamp,year,when,lit,month,upper,lower,initcap,coalesce,trim,expr,regexp_replace
from pyspark.sql import Window
import pandas as pd

# Crear una SparkSession
spark = SparkSession.builder.appName("AzureSQL").getOrCreate()

## Credenciales BD
inpath = "C:/Users/diego.torres/OneDrive/Datasets/Tuboleta/Credenciales.txt"
keys = pd.read_csv(inpath, sep = ',')
display(keys)

# Creo variables para cada fila del DataFrame que contiene las credenciales de la bd
for index, row in keys.iterrows():
    variable_name = row['key']
    variable_value = row['value']
    globals()[variable_name] = variable_value

# Configurar las propiedades de la conexión
jdbc_url = f"jdbc:sqlserver://{jdbc_hostname}:1433;database={jdbc_database_datamart}"
jdbc_properties = {
    "user": user,
    "password": password,
    "driver": driver
}

# Conexion al Blob
spark.conf.set(clave_blob, access_key_blob)

In [ ]:

###############################################
####  AJUSTE LOCALIDAD FISICA

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import re


# Función para convertir abreviaturas de direcciones
def localidad_agrupada(campo):
    if isinstance(campo, str):  # Comprueba si la dirección es una cadena de texto
        #campo = campo.strip()
        # Diccionario de mapeo de abreviaturas
        mapeo_abreviaturas = {
            "PLATEA": "PLATEA",
            "ALFOMBRA": "PLATEA",
            "PALCO": "PLATEA",
            "ORO": "PLATEA",
            "GOLD": "PLATEA",
            "PLATA": "PLATEA",
            "SILVER": "PLATEA",
            "DIAMANTE": "PLATEA",
            "BRONCE": "PLATEA",
            "DANCE": "PLATEA",
            "GENERAL": "PLATEA",
            "BOX": "BOXES",
            "FAN": "TRIBUNAS FAN",
            "SUITE": "SUITES VIP",
            "PISO 2": "PISO 2",
            "201": "PISO 2",
            "P2": "PISO 2",
            "2DO": "PISO 2",
            "FLOOR 2": "PISO 2",
            "SEGUNDO": "PISO 2",
            "PISO 3": "PISO 3",
            "301": "PISO 3",
            "P3": "PISO 3",
            "FLOOR 3": "PISO 3",
            "TERCER": "PISO 3"
            # Puedes agregar más abreviaturas según tus necesidades
        }

        for key in mapeo_abreviaturas.keys():
            if key in campo:
                return mapeo_abreviaturas[key]
        # Unir las palabras de nuevo en una cadena
        return campo
    else:
        return campo


# Registra la función definida por el usuario como un UDF
localidad_agrupada = udf(localidad_agrupada, StringType())

In [ ]:
## MOVEMENTS 2022
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/bronze/Movements/2022/B_Movements_2022/'
movements_2022 = spark.read.parquet(inpath)

## MOVEMENTS 2023
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/bronze/Movements/2023/B_Movements_2023/'
movements_2023 = spark.read.parquet(inpath)

## MOVEMENTS 2024
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/bronze/Movements/2024/B_Movements_2024/'
movements_2024 = spark.read.parquet(inpath)


# SELECCION DE CAMPOS
columnas = ["ORGANIZATION","FILE_NUMBER","FILE_CONTACT","ORDER_NUMBER","ORDER_SALES_CHANNEL","ORDER_DATE","ORDER_CONTACT","SEASON","SITE","PRODUCT_FAMILY","T_PRODUCT_ID","PRODUCT","PRODUCT_DATE_TIME","SEAT_CATEGORY","AUDIENCE_CATEGORY","AUDIENCE_SUB_CATEGORY","AMT_ITX","CHARGE_AMT_ITX","PAYMENT_STATUS","VALIDITY_START_DATE","VALIDITY_END_DATE","T_TICKET_VALIDITY_STATE","T_TICKET_STATE","T_TICKET_PRINT_STATE","TICKET_PRINT_STATE","T_CONTROL_STATUS","CONTROL_STATUS","SHIPMENT_ADDRESS","T_CONTACT_ID","CONTACT","SHIP_ADDR_ZIPCODE","SHIP_ADDR_TOWN","SHIP_ADDR_COUNTRY_CODE","CONTACT_IDENTITY","CONTACT_NATIONALITY","CONTACT_BIRTHDATE","ENTRANCE","PHYSICAL_AREA","PHYSICAL_BLOCK","SEAT_ROW","SEAT_NUMBER","FLOOR","SECTOR","T_SEAT_QUALITY_ID","SEAT_STATE","CONTACT_ADDR_ZIPCODE","CONTACT_ADDR_TOWN","CONTACT_ADDR_COUNTRY_CODE","BASE_UNIT_AMT_ITX","ORDER_CONTACT_NUMBER","PRODUCER_CONTACT_FMT_NAME","LOGICAL_SEAT_CATEGORY","PROMOTION","PROMOTION_TYPE","EXPIRATION_DATE","T_OPERATION_TYPE","OPERATION_TYPE","T_OPERATION_KIND","ORDER_CONTACT_ADDRESS","ORDER_CONTACT_ZIPCODE","ORDER_CONTACT_TOWN","ORDER_CONTACT_COUNTRY","ORIGINATING_SALES_CHANNEL","ORDER_CONTACT_EMAIL","ORDER_CONTACT_PRIMARY_PHONE","T_P_OPERATION_ID","PHYSEAT_ORDER","PHYSEAT_ID"]

movements_2022 = movements_2022.select(columnas)
movements_2023 = movements_2023.select(columnas)


## union de todos los anios
movements_ = movements_2022.union(movements_2023).union(movements_2024)
#print(movements_.count())

26252295


In [ ]:
# tmp = movements_.filter(col('T_PRODUCT_ID').like('%10229415242143%'))\
# #tmp = eventos_movistar.filter(col('Evento').like('%BOOGIE%'))
# display(tmp)

In [ ]:
# tmp2 = movements_.select('LOGICAL_SEAT_CATEGORY','PHYSICAL_AREA').\
#     filter(col('LOGICAL_SEAT_CATEGORY').like('%P2 VISTAS PARCIALES (201 - 219)%')).distinct()
# display(tmp2)

In [ ]:
## Seleccion de columnas ##

columnas = ["ORGANIZATION","FILE_NUMBER","FILE_CONTACT","ORDER_NUMBER","ORDER_SALES_CHANNEL","ORDER_DATE","ORDER_CONTACT","SEASON","SITE","PRODUCT_FAMILY","T_PRODUCT_ID","PRODUCT","PRODUCT_DATE_TIME","SEAT_CATEGORY","AUDIENCE_CATEGORY","AUDIENCE_SUB_CATEGORY","AMT_ITX","CHARGE_AMT_ITX","PAYMENT_STATUS","VALIDITY_START_DATE","VALIDITY_END_DATE","T_TICKET_VALIDITY_STATE","T_TICKET_STATE","T_TICKET_PRINT_STATE","TICKET_PRINT_STATE","T_CONTROL_STATUS","CONTROL_STATUS","SHIPMENT_ADDRESS","T_CONTACT_ID","CONTACT","SHIP_ADDR_ZIPCODE","SHIP_ADDR_TOWN","SHIP_ADDR_COUNTRY_CODE","CONTACT_IDENTITY","CONTACT_NATIONALITY","CONTACT_BIRTHDATE","ENTRANCE","PHYSICAL_AREA","PHYSICAL_BLOCK","SEAT_ROW","SEAT_NUMBER","FLOOR","SECTOR","T_SEAT_QUALITY_ID","SEAT_STATE","CONTACT_ADDR_ZIPCODE","CONTACT_ADDR_TOWN","CONTACT_ADDR_COUNTRY_CODE","BASE_UNIT_AMT_ITX","ORDER_CONTACT_NUMBER","PRODUCER_CONTACT_FMT_NAME","LOGICAL_SEAT_CATEGORY","PROMOTION","PROMOTION_TYPE","EXPIRATION_DATE","T_OPERATION_TYPE","OPERATION_TYPE","T_OPERATION_KIND","ORDER_CONTACT_ADDRESS","ORDER_CONTACT_ZIPCODE","ORDER_CONTACT_TOWN","ORDER_CONTACT_COUNTRY","ORIGINATING_SALES_CHANNEL","ORDER_CONTACT_EMAIL","ORDER_CONTACT_PRIMARY_PHONE","T_P_OPERATION_ID","PHYSEAT_ORDER","PHYSEAT_ID"]

concat_loc = ["LOGICAL_SEAT_CATEGORY",'PHYSICAL_AREA']

### TRANSFORMACIONES ###
movements = movements_.select(columnas).\
    withColumn("LOGICAL_SEAT_CATEGORY", upper("LOGICAL_SEAT_CATEGORY")).\
    withColumn("LOGICAL_SEAT_CATEGORY", regexp_replace(trim(col("LOGICAL_SEAT_CATEGORY")), r'^\s+|\s+$', '')).\
    withColumn("KEY_LOC", concat_ws("|", *concat_loc)).\
    withColumn("LOCALIDAD_GP", localidad_agrupada(col("KEY_LOC"))).\
    withColumn("Fecha Actualizacion",expr("current_timestamp() - interval 5 hours")) # fecha-hora actualizacion
    
    #withColumn("PHYSICAL_AREA", coalesce(col("PHYSICAL_AREA"), col("LOGICAL_SEAT_CATEGORY"))).\


#### MAESTRA DE LOCALIDADES AGRUPADAS POR ZONA FISICA  DEL MOVISTAR ARENA ###
valores = ["PLATEA","BOXES","TRIBUNAS FAN","SUITES VIP","PISO 2",'PISO 3']

# # Aplicación de la lógica de reemplazo en el DataFrame df_follwup
movements = movements.withColumn("LOCALIDAD_GP",
                        when((col("SITE").isin('Movistar Arena - Bogotá - Dg. 61c #26-36')) & 
                        (~col("LOCALIDAD_GP").isin(valores)),
                        lit('PLATEA')).otherwise(col("LOCALIDAD_GP")))

# maestro_localid = movements.filter(col('SITE') == 'Movistar Arena - Bogotá - Dg. 61c #26-36').\
#     select('LOGICAL_SEAT_CATEGORY','LOCALIDAD_GP').\
#     withColumn("LOCALIDAD_GP",
#                 when(~col("LOCALIDAD_GP").isin(valores),
#                 lit('PLATEA')).otherwise(col("LOCALIDAD_GP"))).\
#     distinct()

maestro_localid = movements.filter(col('SITE') == 'Movistar Arena - Bogotá - Dg. 61c #26-36').\
    select('LOGICAL_SEAT_CATEGORY','LOCALIDAD_GP').distinct()


tmp = maestro_localid.select('LOCALIDAD_GP').distinct()
display(tmp)
#print(maestro_localid.count())

#display(tmp)



# tmp = maestro_localid.select('LOCALIDAD_GP').distinct()
# display(tmp)

LOCALIDAD_GP
TRIBUNAS FAN
SUITES VIP
PLATEA
BOXES
PISO 3
PISO 2


In [ ]:
# ########### PRUEBAS #########
# tmp = movements.filter(col('SITE') == 'Movistar Arena - Bogotá - Dg. 61c #26-36').\
#     select('LOGICAL_SEAT_CATEGORY','PHYSICAL_AREA','LOCALIDAD_GP').\
#     distinct()

#     #withColumn("LOCALIDAD_GP", localidad_agrupada(col("LOGICAL_SEAT_CATEGORY"))).\

# print(tmp.count())
# #display(tmp.limit(100))

# display(tmp)

# tmp2 = movements.select('LOGICAL_SEAT_CATEGORY','PHYSICAL_AREA','LOCALIDAD_GP').\
#     filter(col('LOGICAL_SEAT_CATEGORY').like('%P3 (307 A 313)%')).distinct()
# display(tmp2)

# tmp2 = maestro_localid.filter(col('LOGICAL_SEAT_CATEGORY').like('%P3 (307 A 313)%')).distinct()
# display(tmp2)

In [ ]:
# Agregar una columna que indique si es duplicado o no
window_spec = Window().partitionBy("LOGICAL_SEAT_CATEGORY").orderBy("LOGICAL_SEAT_CATEGORY")

df_spark = maestro_localid.withColumn("es_duplicado", count("LOGICAL_SEAT_CATEGORY").over(window_spec) > 1)

# Filtrar solo las filas duplicadas
duplicados = df_spark.filter("es_duplicado")#.select('Evento').distinct()

display(duplicados)
#display(temp.limit(100))

# Obtener los valores distintos de la columna 'LOGICAL_SEAT_CATEGORY' pero manteniendo el dataframe
duplicados = duplicados.select('LOGICAL_SEAT_CATEGORY').distinct()


######  USO LOS DUPLICADOS PARA QUITARLOS DE LA MAESTRA ORIGINAL ####

# Extraer los valores de la columna en una lista
duplicados_list = [row['LOGICAL_SEAT_CATEGORY'] for row in duplicados.collect()]

# quito las localidades con duplicados
maestro_localid = maestro_localid.filter(~col('LOGICAL_SEAT_CATEGORY').isin(duplicados_list))

#print(maestro_localid.count())

LOGICAL_SEAT_CATEGORY,LOCALIDAD_GP,es_duplicado
P3 (307 A 313),PISO 3,true
P3 (307 A 313),PISO 2,true
PARQUEADERO SUITES,BOXES,true
PARQUEADERO SUITES,SUITES VIP,true
PISO 2 (202 - 205)(215 - 218),PISO 2,true
PISO 2 (202 - 205)(215 - 218),PLATEA,true
PISO 2 SEC. 201-219 VISTA PARCIAL,TRIBUNAS FAN,true
PISO 2 SEC. 201-219 VISTA PARCIAL,PISO 2,true
TRIBUNA FAN SUR,TRIBUNAS FAN,true
TRIBUNA FAN SUR,PLATEA,true


1252


In [ ]:
# re aplicamos la funcion de localidad_agrupada
duplicados = duplicados.withColumn("LOCALIDAD_GP", localidad_agrupada(col("LOGICAL_SEAT_CATEGORY")))
display(duplicados.limit(100))

maestro_localid = maestro_localid.union(duplicados).distinct()

LOGICAL_SEAT_CATEGORY,LOCALIDAD_GP
P3 (307 A 313),PISO 3
PARQUEADERO SUITES,SUITES VIP
PISO 2 (202 - 205)(215 - 218),PISO 2
PISO 2 SEC. 201-219 VISTA PARCIAL,PISO 2
TRIBUNA FAN SUR,TRIBUNAS FAN


In [ ]:
# tmp2 = movements.select('LOGICAL_SEAT_CATEGORY','KEY_LOC','PHYSICAL_AREA','LOCALIDAD_GP').\
#     filter(col('LOGICAL_SEAT_CATEGORY').like('%PISO 2 (207 - 213)%')).distinct()
# display(tmp2)


In [ ]:
# df = spark.createDataFrame([
#     (1, "PLATEA 104-106			"),
#     (1, "PLATEA 104-106")
# ], ["id", "LOGICAL_SEAT_CATEGORY"])

# tmp = df.withColumn("LOGICAL_SEAT_CATEGORY_2", trim("LOGICAL_SEAT_CATEGORY")).\
#     select('LOGICAL_SEAT_CATEGORY_2').distinct()
# display(tmp)

In [ ]:
inpath = "abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/Maestras/Localidades_Movistar/"

write_table = maestro_localid.coalesce(1) # coalesce indica que la tabla se va a entregar en una sola particion
write_table.write.csv(inpath, header=True, mode="overwrite", sep='|')

#cambio de nombre
files=dbutils.fs.ls(inpath)
output_file= [x for x in files if x.name.startswith("part-")]

# EN la misma carpeta de origen
dbutils.fs.mv(output_file[0].path, f"{inpath}/localidades_agrupadas_movistar.csv")

True

In [ ]:
# tmp2 = tmp.\
#     select('LOCALIDAD_GP').distinct()

# display(tmp2)

In [ ]:
## AJUSTE PARA QUE SOLO ESCRIBA LO DEL MOVISTAR
movements = movements.filter(col("SITE").isin('Movistar Arena - Bogotá - Dg. 61c #26-36'))
#print(movements.count())

In [ ]:
# Impresion
# Guarda el DataFrame en formato Parquet en Azure Blob Storage
inpath = "abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/silver/Movements/"
# 2023_fcf
join_df_coalesced = movements.coalesce(1)
join_df_coalesced.write.mode("overwrite").parquet(inpath)
#join_df_coalesced.write.mode("overwrite").orc(path_parquet_snappy)

#cambio de nombre
files=dbutils.fs.ls(inpath)
output_file= [x for x in files if x.name.startswith("part-")]

# EN la misma carpeta de origen
dbutils.fs.mv(output_file[0].path, f"{inpath}/Ds_G_movements.parquet")

True